## 전처리한 데이터를 저장, 로드, 디버그할 때 사용!

In [1]:
import io
import os
import json
import distutils.dir_util
from collections import Counter

import numpy as np

# data와 파일 이름을 주면 data폴더 밑에 json파일이 생성!
def write_json(data, fname):
    def _conv(o):
        if isinstance(o, np.int64) or isinstance(o, np.int32): # isinstance: o라는 객체가 numpy배열 int64타입에 속해있는지 확인
            return int(o)
        raise TypeError # 속해있지 않다면 type error발생

    parent = os.path.dirname(fname) # 부모 directory 경로 설정 (fname이 'test/test.json이면 test를 리턴!)
    distutils.dir_util.mkpath("./data/" + parent) # 새로운 경로 만들기(data폴더 밑에 parent 폴더 추가!)
    with io.open("./data/" + fname, "w", encoding="utf8") as f:
        json_str = json.dumps(data, ensure_ascii=False, default=_conv)
        f.write(json_str)


def load_json(fname):
    with open(fname, encoding='utf8') as f:
        json_obj = json.load(f)

    return json_obj


def debug_json(r):
    print(json.dumps(r, ensure_ascii=False, indent=4))

#### Dataset에서 questions / answers 분리하는 함수

In [15]:
# playlist_data를 넣으면 songs에서 랜덤으로 5곡을 골라 questions, answer로 리턴!
import random
import pandas as pd

def split_qna(playlist_data):
    questions = playlist_data.copy()
    answers = pd.DataFrame([], columns=["songs", "id"])
    
    for i in range(len(playlist_data)):
        selected = random.sample(questions['songs'][i], 5) # 리스트에서 5개 랜덤 추출
        answers.loc[i] = [[], questions['id'][i]]
        for j in selected:
            questions['songs'][i].remove(j)
            answers['songs'][i].append(j)

    return questions, answers

#### 사용 예시

In [17]:
import pandas as pd
import random
import import_ipynb

train_playlist = load_json("./data/processedData/playlist_train.json")
train_playlist = pd.DataFrame(eval(train_playlist))
train_playlist.head()

,tags,id,plylst_title,songs,like_cnt,updt_date
0,"[JPOP, 매장음악, 크리스마스, 일본]",24882,jpop 크리스마스 캐롤 등등,"[171751, 25810, 653717, 631818, 486608, 237975...",50,2017-12-17 02:33:47.000
1,"[연말, 겨울감성, 연주곡, 눈, 성탄절, 크리스마스, 설렘, 캐롤, 자장가, 캐롤송]",124080,설레는 크리스마스 캐롤연주곡:),"[220071, 28532, 159001, 329291, 193160, 417231...",8,2018-11-23 23:49:10.000
2,"[겨울, 설렘, 사랑]",15936,☃메리해피✰크리스마스☃,"[404382, 456575, 405143, 478191, 245321, 21731...",0,2014-12-22 12:16:33.000
3,"[겨울, 까페]",24535,크리스마스 카페 음악,"[357064, 137969, 113394, 628504, 205686, 70626...",145,2013-12-04 12:46:13.000
4,"[겨울, 눈오는날, 파티, 카페, 매장, 드라이브, 여행, 크리스마스, 캐롤, 2016]",44592,크리스마스 캐롤 여행을 떠나요,"[355931, 164891, 261338, 52934, 695032, 456704...",15,2016-12-06 01:15:51.000


In [18]:
questions, answers = split_qna(train_playlist)

In [19]:
questions.head()

,tags,id,plylst_title,songs,like_cnt,updt_date
0,"[JPOP, 매장음악, 크리스마스, 일본]",24882,jpop 크리스마스 캐롤 등등,"[171751, 25810, 653717, 631818, 486608, 237975...",50,2017-12-17 02:33:47.000
1,"[연말, 겨울감성, 연주곡, 눈, 성탄절, 크리스마스, 설렘, 캐롤, 자장가, 캐롤송]",124080,설레는 크리스마스 캐롤연주곡:),"[28532, 159001, 193160, 276541, 422708]",8,2018-11-23 23:49:10.000
2,"[겨울, 설렘, 사랑]",15936,☃메리해피✰크리스마스☃,"[404382, 456575, 405143, 217311, 638130, 35818...",0,2014-12-22 12:16:33.000
3,"[겨울, 까페]",24535,크리스마스 카페 음악,"[137969, 205686, 706268, 80723, 695032, 549645...",145,2013-12-04 12:46:13.000
4,"[겨울, 눈오는날, 파티, 카페, 매장, 드라이브, 여행, 크리스마스, 캐롤, 2016]",44592,크리스마스 캐롤 여행을 떠나요,"[355931, 164891, 261338, 52934, 695032, 456704...",15,2016-12-06 01:15:51.000


In [20]:
answers.head()

,songs,id
0,"[381887, 149869, 337430, 361749, 49402]",24882
1,"[220071, 329291, 333474, 417231, 301555]",124080
2,"[238856, 245321, 393865, 478191, 16897]",15936
3,"[628504, 387859, 299708, 357064, 113394]",24535
4,"[345721, 141993, 396550, 465958, 226670]",44592
